# Build Chains and Results  
Instructions: https://fsworks.atlassian.net/wiki/spaces/~558841686/pages/edit-v2/2087550977

In [1]:
instance_name = 'services'
data_file_name = 'EMS_DCMA_DD_23.08.graphml'
experiment = 'services_remote_test_rony'
tasks_types = 'tdas'
results = 'prt'
query = 'code'
if not experiment: experiment = data_file_name.replace('.graphml', '').replace('.', '_')
from modules.run_service import service_manager
service_manager(instance_name, data_file_name, experiment, tasks_types, results, query)

service started on 12:18:07


KeyError: 'services'

# Query Results  

In [ ]:
import duckdb
import os
con = duckdb.connect()
prt_files_path = os.path.join(experiment, 'prt')
results_files = "'{e}/*.parquet'".format(e=prt_files_path)
query = """SELECT * FROM {rf} LIMIT 10""".format(rf=results_files)
query_df = con.execute(query).df()
# To save the query results run:
#query_df.to_excel('query_df.xlsx')
display(query_df)

# Read and Write Chains  

In [ ]:
import pandas as pd
chains_path = os.path.join(experiment, 'chains.parquet')
chains_df = pd.read_parquet(chains_path)
chains = list(chains_df['Chain'])
chains = '\n'.join(chains)+'\n'
with open('chains.txt', 'w') as f: f.write(chains)
display(chains_df.head())
display(chains_df.info())